In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import  mean_squared_error, r2_score
import xgboost as xgb

# import functions
from src.modeling.modeling_functions import error_analysis
from src.preprocessing.preprocessing_functions import drop_rows

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.2f' % x) # change decimal places

RSEED = 42

## Loading the processed dataframe (sugarbeet and field weatherstation data)

In [ ]:
df = pd.read_pickle('pickles/df_fieldweather_devstage_sugarbeet.pkl')
df

In [ ]:
# convert to string for categorizing
df['seednames_coded'] = df['seednames_coded'].astype(str)
df['pollinator_comp'] = df['pollinator_comp'].astype(str)
df['ms_comp'] = df['ms_comp'].astype(str)
df['otype_comp'] = df['otype_comp'].astype(str)

In [ ]:
df.shape

In [ ]:
df.station_location.unique()

In [ ]:
# drop locations for comparison
drop_rows(df, 'station_location', ['Bautzen', 'Goderville', 'Lamotte', 'Lelystad', 'Pithiviers', 
                                   'Rittershausen', 'Sommepy', 'Vierhoefen'])

In [ ]:
# dropping columns 
df.drop(['betaine_nir', 
         'cry_nir', 
         'dm_nir', 
         'invert_nir', 
         'mark_nir', 
         #'sc_nir',
         'csy_nir', 
         'totaln_nir',
         'obj',  
         'seriesid', 
         'x', 
         'y', 
         'ms_comp',
         'otype_comp', 
         #'pollinator_comp',
         #'seednames_coded',
         'region',
         'station_location'
         ], axis=1, inplace=True)
df.columns

In [ ]:
df.shape

In [ ]:
# reset index after drpping columns
df = df.reset_index(drop=True)

#### Creating the pipeline

In [ ]:
# creating list for categorical predictors/features 
cat_features = list(df.columns[df.dtypes==object])
cat_features

In [ ]:
# creating list for numerical predictors/features
# since 'sc_nir' is our target variable we will exclude this feature from the list of numerical predictors 
# latitude and longitude are also excluded to avoid location influence on prediction
num_features = [
 'air_temperature_avg_devstage_1',
 'air_temperature_avg_devstage_2',
 'air_temperature_avg_devstage_3',
 'air_temperature_max_devstage_1',
 'air_temperature_max_devstage_2',
 'air_temperature_max_devstage_3',
 'air_temperature_min_devstage_1',
 'air_temperature_min_devstage_2',
 'air_temperature_min_devstage_3',
 'deltat_avg_devstage_1',
 'deltat_avg_devstage_2',
 'deltat_avg_devstage_3',
 'deltat_max_devstage_1',
 'deltat_max_devstage_2',
 'deltat_max_devstage_3',
 'deltat_min_devstage_1',
 'deltat_min_devstage_2',
 'deltat_min_devstage_3',
 'dew_point_avg_devstage_1',
 'dew_point_avg_devstage_2',
 'dew_point_avg_devstage_3',
 'dew_point_min_devstage_1',
 'dew_point_min_devstage_2',
 'dew_point_min_devstage_3',
 'eag_soil_moisture_1_devstage_1',
 'eag_soil_moisture_1_devstage_2',
 'eag_soil_moisture_1_devstage_3',
 'eag_soil_moisture_2_devstage_1',
 'eag_soil_moisture_2_devstage_2',
 'eag_soil_moisture_2_devstage_3',
 'eag_soil_moisture_3_devstage_1',
 'eag_soil_moisture_3_devstage_2',
 'eag_soil_moisture_3_devstage_3',
 'eag_soil_moisture_4_devstage_1',
 'eag_soil_moisture_4_devstage_2',
 'eag_soil_moisture_4_devstage_3',
 'eag_soil_moisture_5_devstage_1',
 'eag_soil_moisture_5_devstage_2',
 'eag_soil_moisture_5_devstage_3',
 'eag_soil_moisture_6_devstage_1',
 'eag_soil_moisture_6_devstage_2',
 'eag_soil_moisture_6_devstage_3',
 'et0_devstage_1',
 'et0_devstage_2',
 'et0_devstage_3',
#  'latitude_1',
#  'latitude_2',
#  'latitude_3',
 'leaf_wetness_devstage_1',
 'leaf_wetness_devstage_2',
 'leaf_wetness_devstage_3',
#  'longitude_1',
#  'longitude_2',
#  'longitude_3',
 'precipitation_devstage_1',
 'precipitation_devstage_2',
 'precipitation_devstage_3',
 'relative_humidity_avg_devstage_1',
 'relative_humidity_avg_devstage_2',
 'relative_humidity_avg_devstage_3',
 'relative_humidity_max_devstage_1',
 'relative_humidity_max_devstage_2',
 'relative_humidity_max_devstage_3',
 'relative_humidity_min_devstage_1',
 'relative_humidity_min_devstage_2',
 'relative_humidity_min_devstage_3',
 'saturation_vpd_avg_devstage_1',
 'saturation_vpd_avg_devstage_2',
 'saturation_vpd_avg_devstage_3',
 'saturation_vpd_min_devstage_1',
 'saturation_vpd_min_devstage_2',
 'saturation_vpd_min_devstage_3',
 'soil_salinity_1_devstage_1',
 'soil_salinity_1_devstage_2',
 'soil_salinity_1_devstage_3',
 'soil_salinity_2_devstage_1',
 'soil_salinity_2_devstage_2',
 'soil_salinity_2_devstage_3',
 'soil_salinity_3_devstage_1',
 'soil_salinity_3_devstage_2',
 'soil_salinity_3_devstage_3',
 'soil_salinity_4_devstage_1',
 'soil_salinity_4_devstage_2',
 'soil_salinity_4_devstage_3',
 'soil_salinity_5_devstage_1',
 'soil_salinity_5_devstage_2',
 'soil_salinity_5_devstage_3',
 'soil_salinity_6_devstage_1',
 'soil_salinity_6_devstage_2',
 'soil_salinity_6_devstage_3',
 'soil_temperature_1_max_devstage_1',
 'soil_temperature_1_max_devstage_2',
 'soil_temperature_1_max_devstage_3',
 'soil_temperature_1_min_devstage_1',
 'soil_temperature_1_min_devstage_2',
 'soil_temperature_1_min_devstage_3',
 'soil_temperature_1_vg_devstage_1',
 'soil_temperature_1_vg_devstage_2',
 'soil_temperature_1_vg_devstage_3',
 'soil_temperature_2_max_devstage_1',
 'soil_temperature_2_max_devstage_2',
 'soil_temperature_2_max_devstage_3',
 'soil_temperature_2_min_devstage_1',
 'soil_temperature_2_min_devstage_2',
 'soil_temperature_2_min_devstage_3',
 'soil_temperature_2_vg_devstage_1',
 'soil_temperature_2_vg_devstage_2',
 'soil_temperature_2_vg_devstage_3',
 'soil_temperature_3_max_devstage_1',
 'soil_temperature_3_max_devstage_2',
 'soil_temperature_3_max_devstage_3',
 'soil_temperature_3_min_devstage_1',
 'soil_temperature_3_min_devstage_2',
 'soil_temperature_3_min_devstage_3',
 'soil_temperature_3_vg_devstage_1',
 'soil_temperature_3_vg_devstage_2',
 'soil_temperature_3_vg_devstage_3',
 'soil_temperature_4_max_devstage_1',
 'soil_temperature_4_max_devstage_2',
 'soil_temperature_4_max_devstage_3',
 'soil_temperature_4_min_devstage_1',
 'soil_temperature_4_min_devstage_2',
 'soil_temperature_4_min_devstage_3',
 'soil_temperature_4_vg_devstage_1',
 'soil_temperature_4_vg_devstage_2',
 'soil_temperature_4_vg_devstage_3',
 'soil_temperature_5_max_devstage_1',
 'soil_temperature_5_max_devstage_2',
 'soil_temperature_5_max_devstage_3',
 'soil_temperature_5_min_devstage_1',
 'soil_temperature_5_min_devstage_2',
 'soil_temperature_5_min_devstage_3',
 'soil_temperature_5_vg_devstage_1',
 'soil_temperature_5_vg_devstage_2',
 'soil_temperature_5_vg_devstage_3',
 'soil_temperature_6_max_devstage_1',
 'soil_temperature_6_max_devstage_2',
 'soil_temperature_6_max_devstage_3',
 'soil_temperature_6_min_devstage_1',
 'soil_temperature_6_min_devstage_2',
 'soil_temperature_6_min_devstage_3',
 'soil_temperature_6_vg_devstage_1',
 'soil_temperature_6_vg_devstage_2',
 'soil_temperature_6_vg_devstage_3',
 'solar_radiation_devstage_1',
 'solar_radiation_devstage_2',
 'solar_radiation_devstage_3',
 'wind_direction_devstage_1',
 'wind_direction_devstage_2',
 'wind_direction_devstage_3',
 'wind_speed_avg_devstage_1',
 'wind_speed_avg_devstage_2',
 'wind_speed_avg_devstage_3',
 'wind_speed_gusts_devstage_1',
 'wind_speed_gusts_devstage_2',
 'wind_speed_gusts_devstage_3',
 'wind_speed_max_devstage_1',
 'wind_speed_max_devstage_2',
 'wind_speed_max_devstage_3'
]
num_features

#### Train-Test-Split

In [ ]:
# define predictors and target variable
X = df.drop('sc_nir', axis=1)
y = df['sc_nir']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

In [ ]:
# split into train and test set (train set: 70%, test set: 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED) 

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

#### Preprocessing Pipeline

In [ ]:
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Pipeline for numerical features
num_pipeline = Pipeline([
    ('std_scaler', RobustScaler())
])

In [ ]:
# complete pipeline for numerical features
# apply transformers to numerical pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

### Predictive modeling using Pipeline and GridSearch

#### XGBoost

In [ ]:
# building a full pipeline with our preprocessor and the RandomForestRegressor
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', xgb.XGBRegressor(n_estimators = 100))
])

In [ ]:
# making predictions on the training set using cross validation as well as calculating the probabilities
# cross_val_predict expects an estimator (model), X, y and number of cv-splits (cv)
y_train_predicted = cross_val_predict(pipe, X_train, y_train, cv=5)

In [ ]:
# Mean Squared Error
print('MSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Train:\n', r2_score(y_train, y_train_predicted))


#### Optimizing via GridSearch

In [ ]:
# defining parameter space for grid-search
# add 'xgb__' infront of the corresponding hyperparameters
param = {'xgb__max_depth': [10,20,30,40,50],
                  'xgb__max_features': ["sqrt"],
                  'xgb__max_leaf_nodes': [500, 5000],
                  'xgb__min_samples_split': [10, 50, 100]}

grid = GridSearchCV(pipe, param_grid=param, scoring = r2_score, cv=5,
                           verbose=1, n_jobs=-1)

In [ ]:
# train the model
grid.fit(X_train, y_train)

In [ ]:
# show best parameters
print('Best score:\n{:.2f}'.format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

In [ ]:
# save best model (including fitted preprocessing steps) as best_model 
best_model = grid.best_estimator_
best_model

In [ ]:
# calculating the mean squared error, root mean squared error and r^2 score for the test set with the optimized model
y_test_predicted = best_model.predict(X_test)

# Mean Squared Error
print('MSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Test:\n', r2_score(y_test, y_test_predicted))

In [ ]:
# evaluate the error
error_analysis(y_test,y_test_predicted)

#### Feature importance

In [ ]:
# fit pipe with train data
model = pipe.fit(X_train, y_train)

In [ ]:
# get coefficients
coefs = model.steps[1][1].feature_importances_
coefs

In [ ]:
# get feature names
feature_names = model.steps[0][1].get_feature_names_out()

In [ ]:
# zip coefficients and names together and make a DataFrame
zipped = zip(feature_names, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "#2f7055" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)

In [ ]:
zipped_field = df
zipped_field

In [ ]:
zipped_field.to_csv('XGBoost_field_coefficients.csv')

In [ ]:
# plot feature importance
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.barplot(x="feature",
            y="value",
            data=df.head(5),
           palette=df.head(5)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=20)
ax.set_title("Top 5 Features", fontsize=25)
ax.set_ylabel("Coef", fontsize=22)
ax.set_xlabel("Feature Name", fontsize=22)

## Loading the processed dataframe (sugarbeet and openweather weatherstation data)

In [ ]:
df = pd.read_pickle('pickles/df_openweather_devstage_sugarbeet.pkl')
df

In [ ]:
# convert to string for categorizing
df['seednames_coded'] = df['seednames_coded'].astype(str)
df['pollinator_comp'] = df['pollinator_comp'].astype(str)
df['ms_comp'] = df['ms_comp'].astype(str)
df['otype_comp'] = df['otype_comp'].astype(str)

In [ ]:
df.station_location.unique()

In [ ]:
# drop locations for comparison
drop_rows(df, 'station_location', ['Bautzen', 'Goderville', 'Lamotte', 'Lelystad', 'Pithiviers', 
                                   'Rittershausen', 'Sommepy', 'Vierhoefen'])

In [ ]:
df.shape

In [ ]:
# dropping columns 
df.drop(['betaine_nir', 
         'cry_nir', 
         'dm_nir', 
         'invert_nir', 
         'mark_nir', 
         #'sc_nir',
         'csy_nir', 
         'totaln_nir',
         'obj',  
         'seriesid', 
         'x', 
         'y', 
         'ms_comp',
         'otype_comp', 
         #'pollinator_comp',
         #'seednames_coded',
         'region',
         'station_location'
         ], axis=1, inplace=True)
df.columns

In [ ]:
df.shape

In [ ]:
# reset index after drpping columns
df = df.reset_index(drop=True)

#### Creating the pipeline

In [ ]:
# creating list for categorical predictors/features 
cat_features = list(df.columns[df.dtypes==object])
cat_features

In [ ]:
# creating list for numerical predictors/features
# since 'sc_nir' is our target variable we will exclude this feature from the list of numerical predictors 
# latitude and longitude were previously excluded to avoid location influence on prediction
num_features = [
 'dew_point_dev_stage_1', 
 'dew_point_dev_stage_2',
 'dew_point_dev_stage_3', 
 'humidity_dev_stage_1', 
 'humidity_dev_stage_2',
 'humidity_dev_stage_3', 
 'pressure_dev_stage_1', 
 'pressure_dev_stage_2',
 'pressure_dev_stage_3', 
 'temp_dev_stage_1', 
 'temp_dev_stage_2',
 'temp_dev_stage_3', 
 'temp_max_dev_stage_1', 
 'temp_max_dev_stage_2',
 'temp_max_dev_stage_3', 
 'temp_min_dev_stage_1', 
 'temp_min_dev_stage_2',
 'temp_min_dev_stage_3', 
 'wind_deg_dev_stage_1', 
 'wind_deg_dev_stage_2',
 'wind_deg_dev_stage_3',
 'wind_speed_dev_stage_1',
 'wind_speed_dev_stage_2', 
 'wind_speed_dev_stage_3'
]
num_features

#### Trian-Test-Split

In [ ]:
# define predictors and target variable
X = df.drop('sc_nir', axis=1)
y = df['sc_nir']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

In [ ]:
# split into train and test set (train set: 70%, test set: 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED) 

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

#### Preprocessing Pipeline

In [ ]:
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Pipeline for numerical features
num_pipeline = Pipeline([
    ('std_scaler', RobustScaler())
])

In [ ]:
# complete pipeline for numerical features
# apply transformers to numerical pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

### Predictive modeling using Pipeline and GridSearch

#### XGBoost

In [ ]:
# building a full pipeline with our preprocessor and the RandomForestRegressor
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', xgb.XGBRegressor(n_estimators = 100))
])

In [ ]:
# making predictions on the training set using cross validation as well as calculating the probabilities
# cross_val_predict expects an estimator (model), X, y and number of cv-splits (cv)
y_train_predicted = cross_val_predict(pipe, X_train, y_train, cv=5)

In [ ]:
# Mean Squared Error
print('MSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Train:\n', r2_score(y_train, y_train_predicted))

#### Optimizing via GridSearch

In [ ]:
# defining parameter space for grid-search
# add 'randomforest__' infront of the corresponding hyperparameters
param = {'xgb__max_depth': [10,20,30,40,50],
                  'xgb__max_features': ["sqrt"],
                  'xgb__max_leaf_nodes': [500, 5000],
                  'xgb__min_samples_split': [10, 50, 100]}

grid = GridSearchCV(pipe, param_grid=param, scoring = r2_score, cv=5,
                           verbose=1, n_jobs=-1)

In [ ]:
# train the model
grid.fit(X_train, y_train)

In [ ]:
# show best parameters
print('Best score:\n{:.2f}'.format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

In [ ]:
# save best model (including fitted preprocessing steps) as best_model 
best_model = grid.best_estimator_
best_model

In [ ]:
# calculating the mean squared error, root mean squared error and r^2 score for the test set with the optimized model
y_test_predicted = best_model.predict(X_test)

# Mean Squared Error
print('MSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Test:\n', r2_score(y_test, y_test_predicted))

In [ ]:
# evaluate the error
error_analysis(y_test,y_test_predicted)

#### Feature importance

In [ ]:
# fit pipe with train data
model = pipe.fit(X_train, y_train)

In [ ]:
# get coefficients
coefs = model.steps[1][1].feature_importances_
coefs

In [ ]:
# get feature names
feature_names = model.steps[0][1].get_feature_names_out()

In [ ]:
# zip coefficients and names together and make a DataFrame
zipped = zip(feature_names, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "#2f7055" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)

In [ ]:
zipped_openweather = df
zipped_openweather

In [ ]:
zipped_openweather.to_csv('XGBoost_openweather_coefficients.csv')

In [ ]:
# plot feature importance
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.barplot(x="feature",
            y="value",
            data=df.head(5),
           palette=df.head(5)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=20)
ax.set_title("Top 5 Features", fontsize=25)
ax.set_ylabel("Coef", fontsize=22)
ax.set_xlabel("Feature Name", fontsize=22)